In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2

In [ ]:
X_train, train_y, X_test, test_y = [],[],[],[]

In [ ]:
df = pd.read_csv('/content/fer2013.csv')

In [ ]:
for index, row in df.iterrows():
  val = row['pixels'].split(" ")
  try:
    if 'Training' in row['Usage']:
      X_train.append(np.array(val, 'float32'))
      train_y.append(row['emotion'])
    elif 'PublicTest' in row['Usage']:
      X_test.append(np.array(val, 'float32'))
      test_y.append(row['emotion'])
  except:
    print(f"error accured at index : {index} and row : {row}")

In [ ]:
num_features = 64
num_labels = 7
batsh_size = 64
epochs = 200
width, height = 48, 48

In [ ]:
X_train = np.array(X_train, 'float32')
train_y = np.array(train_y, 'float32')
X_test = np.array(X_test, 'float32')
test_y = np.array(test_y, 'float32')

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
train_y = to_categorical(train_y, num_classes=num_labels)
test_y = to_categorical(test_y, num_classes=num_labels)

In [ ]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))


In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, train_y,
          batch_size=batsh_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

In [ ]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
  json_file.write(fer_json)
model.save_weights("fer.h5")

In [ ]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image


In [ ]:
model = model_from_json(open("fer.json", "r").read())
model.load_weights('fer.h5')

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
test_image = cv2.imread('/content/sadem.jpg')
gray_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray_image)

In [ ]:
from keras.preprocessing.image import img_to_array
face_haar_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
faces = face_haar_cascade.detectMultiScale(gray_image, 1.1, 4)
for(x ,y ,w, h) in faces:
  cv2.rectangle(test_image, (x, y), (x+w, y+h), (255,0,0))
  roi_gray = gray_image[y:y+w, x:x+h]
  roi_gray = cv2.resize(roi_gray, (48,48))
  image_pixels = img_to_array(roi_gray)
  image_pixels = np.expand_dims(image_pixels, axis=0)
  image_pixels /= 255
  predictions = model.predict(image_pixels)
  max_index = np.argmax(predictions[0])
  emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
  emotion_prediction = emotion_detection[max_index]
  print(emotion_prediction)

  font = cv2.FONT_HERSHEY_SIMPLEX
  org = (50,50)
  frontScale = 1
  color = (255, 0, 0)
  thickness = 2

  image = cv2.putText(test_image, emotion_prediction, org, font, 
                      frontScale, color, thickness, cv2.LINE_AA)
  cv2_imshow(image)
